In [ ]:
from tensorflow.keras.layers import Conv2D, Dropout , Concatenate , SpatialDropout2D , GlobalAveragePooling2D  , Normalization, BatchNormalization , Activation   , AveragePooling2D  , MaxPooling2D, ZeroPadding2D , Dense , Add , ZeroPadding2D  , Input ,Flatten , Reshape
from tensorflow.keras.models import Model , Sequential
import tensorflow as tf 
import tensorflow.keras.backend as K 
from tensorflow.keras import mixed_precision
from tensorflow.keras.regularizers import L2
from tensorflow.keras.callbacks import ReduceLROnPlateau
import gc
from sklearn.model_selection import train_test_split
mixed_precision.set_global_policy('float32')
IMG_SIZE = 32
import matplotlib.pyplot as plt


label_mapping = {0:'T-shirt/top',
               1: 'Trouser',
               2: 'Pullover',
               3: 'Dress',
               4: 'Coat',
               5: 'Sandal',
               6: 'Shirt',
               7: 'Sneaker',
               8: 'Bag',
               9: 'Ankle Boot'}
def plotlearningcurve(history):
    plt.figure(figsize = (16,9))
    plt.subplot(121)
    plt.plot(history.history['categorical_accuracy'] , label = 'Train Accuracy')
    plt.plot(history.history['val_categorical_accuracy'] , label = 'Validation Accuracy')
    plt.title('Learning Curve - Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.subplot(122)
    plt.plot(history.history['loss'] , label = 'Train Accuracy')
    plt.plot(history.history['val_loss'] , label = 'Validation Accuracy')
    plt.title('Learning Curve - loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')


    plt.legend( loc='upper left')
    plt.show()

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
import numpy as np 
def class_analysis(model, X_val , y_val):
    finalpredictions = model.predict(X_val)
   
    finalpredictions_final=np.argmax(finalpredictions, axis=1)
    y_test_final=np.argmax(y_val, axis=1) if y_val.ndim == 2 else y_val
    classificationresults = classification_report(y_test_final ,finalpredictions_final  , target_names = label_mapping.values() , output_dict =True)
    print('Accuracy' , accuracy_score(y_test_final ,finalpredictions_final ))
    return pd.DataFrame(classificationresults).T.sort_values('f1-score')



In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
x_train = x_train/255.0
x_test = x_test/255.0

# x_train = x_train.reshape( (*x_train.shape , 1))
# x_test = x_test.reshape( (*x_test.shape , 1))

y_train= tf.keras.utils.to_categorical(y_train, num_classes=100)
y_test= tf.keras.utils.to_categorical(y_test, num_classes=100)




train_and_val_size = len(x_train) 

print(train_and_val_size)
trainsize= int(0.8*train_and_val_size)

test = tf.data.Dataset.from_tensor_slices((x_test ,y_test ))

X_actual_train, X_val, y_actual_train, y_val = train_test_split(  x_train, y_train, test_size=0.2, random_state=42)

train = tf.data.Dataset.from_tensor_slices((X_actual_train ,y_actual_train ))
val = tf.data.Dataset.from_tensor_slices((X_val ,y_val ))

def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.image.convert_image_dtype(image, tf.float32) 
    return image, label



simpleaugpipeline = Sequential([
    tf.keras.layers.RandomCrop(IMG_SIZE, IMG_SIZE),
    tf.keras.layers.RandomFlip(mode = 'horizontal'),
    tf.keras.layers.RandomTranslation(0.2, 0.2),
    tf.keras.layers.RandomZoom(0.2,0.2)

])


train = train.map(preprocess_image)

# train_augmented = train.map(lambda img,label: (simpleaugpipeline(img) , label), num_parallel_calls=tf.data.AUTOTUNE)


                                
def helper(*datasets):
    imgs = [i[0] for i in datasets]
    labels = [i[1] for i in datasets]

    return tf.stack(imgs), tf.stack(labels)

# fi

    # for img,label in datasets:
    #     print('img',img, 'label',label)
    # return datasets
datasets_zipped = tf.data.Dataset.zip((train,))
datasets_noaug = datasets_zipped.map(helper).unbatch().map(preprocess_image).batch(128).prefetch(tf.data.AUTOTUNE)



50000


VGG + No Augmentation

### VGG with simple augmentation 
1. `RandomCrop(IMG_SIZE, IMG_SIZE)` 
2. `RandomFlip(mode = 'horizontal')`
3. `RandomTranslation(0.2, 0.2)`
4. `RandomZoom(0.2,0.2)`



> To Provide more stability to training
- Dropout is decreased to 0.2 
- L2 regularisation/ weight decay decrease to 0.0005 

In [ ]:

def vggblock(x , n_conv, n_filters):
    for _ in range(n_conv):
        x = Conv2D(filters=n_filters,kernel_size=(3,3),padding="same", activation="relu" , kernel_regularizer = L2( 0.0005))(x)
        x= BatchNormalization()(x)
    return x
def createcustomvgg():
    inputtensor = Input(shape=(32,32,3))
    x = vggblock(inputtensor, 2, 32)
    x= MaxPooling2D((2,2) , strides =2 )(x)
    x = SpatialDropout2D(0.2)(x)
    x = vggblock(x , 2 , 64)
    x= MaxPooling2D((2,2) , strides =2 )(x)
    x = SpatialDropout2D(0.2)(x)
    x = vggblock(x , 2 , 128)
    x =  MaxPooling2D((2,2) , strides =2 )(x)
    x = SpatialDropout2D(0.2)(x)
    x = vggblock(x , 2 , 256)
    x = MaxPooling2D((2,2) , strides =2 )(x)
    x = SpatialDropout2D(0.2)(x)
    x = vggblock(x, 2, 256)
    x = GlobalAveragePooling2D()(x)
    x= Dense(100 , 'softmax')(x)
    return Model( inputs = [inputtensor] , outputs = [x])

vggreg2 = createcustomvgg()
vggreg2.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_30 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_30 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_31 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_31 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 16, 16, 32)       0   

## Cutmix Augmentation

In [ ]:

IMG_SIZE =32
def preprocess_image(image, label):
    image = tf.image.resize(image, (32, 32))
    image = tf.image.convert_image_dtype(image, tf.float32) 
    return image, label


@tf.function
def sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)


@tf.function
def get_box(lambda_value):
    cut_rat = tf.math.sqrt(1.0 - lambda_value)

    cut_w = IMG_SIZE * cut_rat  # rw
    cut_w = tf.cast(cut_w, tf.int32)

    cut_h = IMG_SIZE * cut_rat  # rh
    cut_h = tf.cast(cut_h, tf.int32)

    cut_x = tf.random.uniform((1,), minval=0, maxval=IMG_SIZE, dtype=tf.int32)  # rx
    cut_y = tf.random.uniform((1,), minval=0, maxval=IMG_SIZE, dtype=tf.int32)  # ry

    boundaryx1 = tf.clip_by_value(cut_x[0] - cut_w // 2, 0, IMG_SIZE)
    boundaryy1 = tf.clip_by_value(cut_y[0] - cut_h // 2, 0, IMG_SIZE)
    bbx2 = tf.clip_by_value(cut_x[0] + cut_w // 2, 0, IMG_SIZE)
    bby2 = tf.clip_by_value(cut_y[0] + cut_h // 2, 0, IMG_SIZE)

    target_h = bby2 - boundaryy1

    target_h_new = tf.cond(target_h ==0 , lambda : target_h+1,  lambda :target_h)
    print(type(target_h))
    # if target_h == 0:
    #     target_h += 1

    target_w = bbx2 - boundaryx1
    target_w_new = tf.cond(target_w ==0 , lambda:target_w+1, lambda:target_w)
    print(target_w)

    # if target_w == 0:
    #     target_w += 1

    return boundaryx1, boundaryy1, target_h_new, target_w_new


@tf.function
def cutmix(train_ds_one, train_ds_two):
    (image1, label1), (image2, label2) = train_ds_one, train_ds_two
    # image1 = train_ds_one[0]
    # label1 = train_ds_one[1]
    # image2=train_ds_two[0]
    # label2 =train_ds_two[1]

    alpha = [1]
    beta = [1]

    # Get a sample from the Beta distribution
    lambda_value = sample_beta_distribution(1, alpha, beta)

    # Define Lambda
    lambda_value = lambda_value[0][0]

    # Get the bounding box offsets, heights and widths
    boundaryx1, boundaryy1, target_h, target_w = get_box(lambda_value)

    # Get a patch from the second image (`image2`)
    crop2 = tf.image.crop_to_bounding_box(
        image2, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image2` patch (`crop2`) with the same offset
    image2 = tf.image.pad_to_bounding_box(
        crop2, boundaryy1, boundaryx1, IMG_SIZE, IMG_SIZE
    )
    # Get a patch from the first image (`image1`)
    crop1 = tf.image.crop_to_bounding_box(
        image1, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image1` patch (`crop1`) with the same offset
    img1 = tf.image.pad_to_bounding_box(
        crop1, boundaryy1, boundaryx1, IMG_SIZE, IMG_SIZE
    )

    # Modify the first image by subtracting the patch from `image1`
    # (before applying the `image2` patch)
    image1 = image1 - img1
    # Add the modified `image1` and `image2`  together to get the CutMix image
    image = image1 + image2

    # Adjust Lambda in accordance to the pixel ration
    lambda_value = 1 - (target_w * target_h) / (IMG_SIZE * IMG_SIZE)
    lambda_value = tf.cast(lambda_value, tf.float32)

    # Combine the labels of both images
    label = lambda_value * label1 + (1 - lambda_value) * label2
    return image, label


In [ ]:
# tf.config.run_functions_eagerly(False)

vggreg3 = tf.keras.models.clone_model(vggreg2)

# Compile model
vggreg3.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum = 0.9 ),
    metrics=['categorical_accuracy'],
)
train_augmented = tf.data.Dataset.zip((train.shuffle(2048) , train.shuffle(2048))).map(cutmix, num_parallel_calls=tf.data.AUTOTUNE)

datasets_zipped = tf.data.Dataset.zip((train_augmented, ))
datasets_noaug = datasets_zipped.map(helper).unbatch().batch(128).prefetch(tf.data.AUTOTUNE)

history2 = vggreg3.fit(datasets_noaug,
          epochs=100,
          validation_data = val.batch(128), 
          callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_categorical_accuracy' , patience = 10 , restore_best_weights = True  )]
          )

<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("sub_4:0", shape=(), dtype=int32)
Epoch 1/100
313/313 [==============================] - 22s 55ms/step - loss: 4.9741 - categorical_accuracy: 0.0586 - val_loss: 4.8364 - val_categorical_accuracy: 0.0620
Epoch 2/100
313/313 [==============================] - 20s 63ms/step - loss: 4.7026 - categorical_accuracy: 0.1023 - val_loss: 4.1557 - val_categorical_accuracy: 0.1613
Epoch 3/100
313/313 [==============================] - 17s 53ms/step - loss: 4.5540 - categorical_accuracy: 0.1231 - val_loss: 3.8610 - val_categorical_accuracy: 0.2131
Epoch 4/100
313/313 [==============================] - 18s 56ms/step - loss: 4.4366 - categorical_accuracy: 0.1467 - val_loss: 3.7167 - val_categorical_accuracy: 0.2320
Epoch 5/100
313/313 [==============================] - 17s 54ms/step - loss: 4.3326 - categorical_accuracy: 0.1647 - val_loss: 3.6354 - val_categorical_accuracy: 0.2400
Epoch 6/100
313/313 [==============================] - 18s 56ms/s

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
vggreg3.save('vggmodel')

In [ ]:
import pytest
import requests
import base64
import json
from tensorflow.keras.datasets.cifar100 import load_data
import numpy as np
#load MNIST dataset
(_, _), (x_test, y_test) = load_data()
# reshape data to have a single channel
# x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2],
# 1))
# normalize pixel values
x_test = x_test.astype('float32') / 255.0
#server URL
import tensorflow as tf
vgg = tf.keras.models.load_model('vggmodel')
url = 'https://ca2-tensorflow-serving-model-deployment.onrender.com/v1/models/vgg:predict' #see [B]
def make_prediction(instances):
    data = json.dumps({"signature_name": "serving_default",
    "instances": instances.tolist()}) #see [C]
    headers = {"content-type": "application/json"}
    json_response = requests.post(url, data=data, headers=headers)
    
    print(json_response.text)
    predictions = json.loads(json_response.text)['predictions']
    return predictions
def test_prediction():
    predictions = make_prediction(x_test[0:4]) #see [A]
    print(predictions)
    # for i, pred in enumerate(predictions):
    #     assert y_test[i] == np.argmax(pred) #see [D]
test_prediction() 

1/1 [==============================] - 0s 229ms/step
[[1.92493157e-04 9.58798119e-05 1.25727023e-03 8.20443325e-04
  1.08714250e-03 2.00509792e-04 5.23703988e-04 1.58278213e-03
  2.18193326e-03 1.15511146e-04 2.91189819e-04 1.77381048e-03
  3.71770747e-02 5.36759151e-04 4.18511714e-04 1.74854079e-03
  2.29002166e-04 1.50766559e-02 7.52128719e-04 5.29776909e-04
  1.86728299e-04 9.53660696e-04 8.53074074e-04 2.23987512e-02
  2.04341509e-03 8.00129608e-04 2.51153484e-03 1.10829214e-03
  7.82754621e-04 1.25557173e-03 5.68836220e-02 7.53687869e-04
  5.06190350e-04 6.98749558e-04 4.83619806e-04 9.46124201e-04
  6.72343071e-04 9.09417402e-03 1.16725848e-03 4.34757530e-04
  1.49452104e-03 1.64703553e-04 4.93756204e-04 7.72553263e-04
  1.11211743e-02 4.14603070e-04 1.49394327e-03 3.51985567e-04
  1.15482521e-03 9.90608707e-02 3.46994144e-03 1.87983765e-04
  3.49691109e-04 1.53004366e-04 2.57235835e-04 4.04803827e-03
  3.74925067e-03 6.06349262e-04 3.11662763e-04 3.94965429e-03
  4.90245875e-03 

In [ ]:
vggreg3.save('vggmodel')

In [ ]:
model = tf.keras.models.load_model('vggmodel')

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0     